In [212]:
import datetime

In [213]:
today = datetime.datetime.today()

In [214]:
residents = [
    ("senior51", 5),
    ("senior52", 5),
    ("senior41", 4),
    ("senior42", 4),
    ("junior31", 3),
    ("junior32", 3),
    ("junior21", 2),
    ("junior22", 2)
]

In [215]:
seniors = [x for x in residents if x[1] in (4,5)]
juniors = [x for x in residents if x[1] in (2,3)]

In [216]:
def rand_from_l(l):
    index = int(random.random() * len(l))
    return l[index]

In [217]:
def is_fri_sat_sun(day):
    return day.weekday() in (4,5,6)
def is_fri_sun(day):
    return day.weekday() in (4,6)

def mon_thru_fri(day):
    return day.weekday() in (0,1,2,3,4)

In [218]:
today = datetime.datetime(year=2019, month=7, day=1)

In [232]:
import random

holidays = {
    "thanks": {
        "days": [datetime.datetime(year=2019, month=11, day=28) + datetime.timedelta(days=x) for x in range(4)],
        "senior": [('senior41', 4), ('senior41', 4), ('senior41', 4), ('senior41', 4)],
        "junior": [('junior21', 2), ('junior21', 2), ('junior22', 2), ('junior22', 2)]
    },
    #"july4": None,
    #"memorial": None,
    #"labor": None
}
thanksgiving = []

def is_holiday(day):
    for k,v in holidays.items():
        if day in v['days']:
            my_index = v['days'].index(day)
            return v['senior'][my_index], v['junior'][my_index]
    return None

def random_call_schedule():
    today = datetime.datetime(2019, 7, 1, 0, 0)
    weekend_senior = seniors[0]
    fri_sun_junior = None
    calls = []
    for i in range(182):
        call_senior = rand_from_l(seniors)
        call_junior = rand_from_l(juniors)
        day = today + datetime.timedelta(days=i)
        retval = is_holiday(day)
        if retval is not None:
            call_senior, call_junior = retval
            calls.append([day, call_senior, call_junior])
            continue
        
        if mon_thru_fri(day):
            if len(calls) != 0:
                last_senior = calls[-1][1]
                last_junior = calls[-1][2]
                while call_junior == last_junior:
                    call_junior = rand_from_l(juniors)
                while call_senior == last_senior:
                    call_senior = rand_from_l(seniors)
        
        if day.weekday() == 4:
            weekend_senior = call_senior
            fri_sun_junior = call_junior
            
        if is_fri_sat_sun(day):
            call_senior = weekend_senior
        if is_fri_sun(day):
            call_junior = fri_sun_junior
        while day.weekday() == 5 and call_junior == fri_sun_junior:
            call_junior = rand_from_l(juniors)

        calls.append([day, call_senior, call_junior])
    return calls

In [233]:
def count_totals(calls):
    names = {}
    for elem in calls:
        senior = elem[1][0]
        junior = elem[2][0]
        if senior not in names:
            names[senior] = 0
        if junior not in names:
            names[junior] = 0
        names[senior] += 1
        names[junior] += 1
    return names

def count_weekends(calls):
    names = {}
    for elem in calls:
        day_of_week = elem[0].weekday()
        if day_of_week not in (5,6):
            continue
        senior = elem[1][0]
        junior = elem[2][0]
        if senior not in names:
            names[senior] = 0
        if junior not in names:
            names[junior] = 0
        names[senior] += 1
        names[junior] += 1
    return names

In [234]:
def score_it(calls):
    totals = count_totals(calls)
    values = list(totals.values())
    p1 = max(values) - min(values)
    
    totals = count_weekends(calls)
    values = list(totals.values())
    p2 = max(values) - min(values)
    return p1 + p2

In [235]:
def get_best_schedule(n_bootstraps=1000):
    best_call = random_call_schedule()
    for i in range(n_bootstraps):
        my_call = random_call_schedule()
        my_score = score_it(calls)
        if score_it(my_call) < score_it(best_call):
            best_call = my_call
    return best_call

In [236]:
import time
start_time = datetime.datetime.now()
best_call = get_best_schedule(100000)
end_time = datetime.datetime.now()
print(end_time - start_time)

0:01:22.210087


In [237]:
dows = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday"
}
def print_call(calls, person=None):
    if person is None:
        people = [x[0] for x in residents]
    else:
        people = [person]
    for c in calls:
        if c[1][0] in people or c[2][0] in people:
            dow = dows[c[0].weekday()]
            print([dow] + [str(x) for x in c])

In [238]:
print_call(best_call)

['Monday', '2019-07-01 00:00:00', "('senior52', 5)", "('junior21', 2)"]
['Tuesday', '2019-07-02 00:00:00', "('senior41', 4)", "('junior22', 2)"]
['Wednesday', '2019-07-03 00:00:00', "('senior51', 5)", "('junior32', 3)"]
['Thursday', '2019-07-04 00:00:00', "('senior41', 4)", "('junior22', 2)"]
['Friday', '2019-07-05 00:00:00', "('senior42', 4)", "('junior21', 2)"]
['Saturday', '2019-07-06 00:00:00', "('senior42', 4)", "('junior31', 3)"]
['Sunday', '2019-07-07 00:00:00', "('senior42', 4)", "('junior21', 2)"]
['Monday', '2019-07-08 00:00:00', "('senior51', 5)", "('junior31', 3)"]
['Tuesday', '2019-07-09 00:00:00', "('senior52', 5)", "('junior32', 3)"]
['Wednesday', '2019-07-10 00:00:00', "('senior51', 5)", "('junior21', 2)"]
['Thursday', '2019-07-11 00:00:00', "('senior42', 4)", "('junior22', 2)"]
['Friday', '2019-07-12 00:00:00', "('senior41', 4)", "('junior21', 2)"]
['Saturday', '2019-07-13 00:00:00', "('senior41', 4)", "('junior22', 2)"]
['Sunday', '2019-07-14 00:00:00', "('senior41', 